<center>
<h1 style="color:red; font-size:70px;"> Simple CNN</h1>

In questo jupiter notebook è presente un semplice esempio di CNN classifier, il cui obiettivo è quello di riconoscere se le radiografie polmonari di alcuni pazienti evidenziano l'infezione da COVID-19 oppure no. Il dataset in uso è stato scaricato da Kaggle al seguente link: https://www.kaggle.com/datasets/khoongweihao/covid19-xray-dataset-train-test-sets?resource=download.

In [8]:
from tensorflow import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Flatten, Dense, Rescaling, MaxPooling2D, Conv2D, Dropout

## Costruzione del dataset

In questo spezzone di codice viene preparato il dataset di immagini perché possa essere utilizzato in maniera semplice da una rete neurale convoluzionale. Per prima cosa viene specificato dove si trovano i le immagini; viene fornito solo il nome della cartella madre, perchè le sottocartelel saranno interpretate come le varie classi. Una volta normalizzati i valori dei pixels, si utilizza la funzione ***flow_from_directory*** per leggere le immagini e ridimensionarle alla size desiderata. Inoltre, dato che viene specificata una batch size di 32, le immagini non vengono caricate tutte insieme in memoria, ma elaborate a piccoli gruppi.

In [7]:
# Raccoglie le varie classi, che avranno come nome quello delle sotto-cartelle
dataset_dir = "Datasets/satellite"  

# Generatore con split per validation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  
)

# Dataset di training
trDat = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

# Dataset di validation
valDat = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical",
    subset="validation",
    shuffle=True
)

Found 4505 images belonging to 4 classes.
Found 1126 images belonging to 4 classes.


## Costruzione del modello

Le reti neurali convoluzionali (CNN, Convolutional Neural Networks) prendono il loro nome dal fatto che il loro elemento fondamentale è l’operazione di convoluzione. In altre parole, invece di collegare ogni neurone a tutti i pixel di un’immagine come nelle reti dense tradizionali, le CNN applicano filtri (kernel) piccoli che scorrono sull’immagine per rilevare caratteristiche locali come bordi, angoli, texture o pattern specifici.

Questa logica è proprio quella implementata nella funzione ***buildMod***, il cui obiettivo è quello di restituire un modello che consenta un'analisi accurata delle immagini con conseguente classificazione.

In [11]:
# Funzione per costruire il modello in questione
def builMod(imH, imW, numcl):

    model = keras.Sequential()
    model.add(Input(shape=(imH, imW, 3)))
    # Per i layer convoluzionali utilizzaimo Conv2D, che crea un kernel convoluzionale che è in 
    # convoluzione con l'input su uno spazio 2D (larghezza e lunghezza). In particolare specifichiamo:
    #
    # filters --> il numero di filtri da applicare. Ogni filtro si dedica all'individuazione di 
    # una caratteristica differente, quindi "più filtri == più caratteristiche 
    # imparate"
    #
    # kernel_size --> specifica le dimensioni del filtro, quindi larghezza ed altezza
    #
    # Si possono anche specificare altri parametri quali la funzione d'attivazione
    model.add(Conv2D(16, (3, 3), activation = 'relu'))
    # Dopo uno strato convozionale introduciamo un layer di pooling, il cui scopo è quello di 
    # rendere più efficiente la rete neurale, dato che la dimensione delle immagini vinee ridotta, 
    # mantenendo tuttavia le caratteristiche principali. Questo vuol dire che le successive analisi 
    # saranno più immmediate. Un parametro importante da segnalare è il "pool_size", che definisce 
    # le dimensioni della finestra di campionamento
    model.add(MaxPooling2D((2, 2)))
    # Per rendere lo studio dell'immagine più accurato inserisco altri layer convoluzionali e di 
    # pooling, in modo da caratterizzare altre features
    model.add(Conv2D(32, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation = 'relu'))

    # Inizia ora la parte fully connected per la classificazione delle immagini
    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(numcl, activation = 'softmax'))
    
    return model

# Costruzione del modello ed allenamento della rete
model = builMod(32, 32, 10)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

2025-08-27 18:58:14.444344: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
